In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob
from datetime import datetime as dt
import seaborn as sns

np.set_printoptions(suppress=True)

%matplotlib inline

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [2]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

## load 2017 data plus additional manipulations

In [94]:
df_sf_2017 = pd.read_json('df_sf_2017')
df_sf_2017.loc[df_sf_2017.cancellation_policy == 'flexible_new', 'cancellation_policy'] = "flexible"
prop_type = set(df_sf_2017.property_type.values)
keep = ['Apartment', 'House', 'Condomonium']
drop = list(set(prop_type).difference(keep))
df_sf_2017['property_type_new'] = df_sf_2017['property_type']
df_sf_2017.loc[df_sf_2017.property_type.isin(drop), 'property_type_new'] = "Other"

In [54]:
# df_sf['price'].apply(np.log)[df_sf_2017['price_per_guest'].isnull()][[ 'price', 'guests_included','price_per_guest']]

,price,guests_included,price_per_guest


In [81]:
df_sf_2017.price_per_bedroom.isnull().sum()

0

## Random Forest

In [95]:
def grid_searching (param_grid, model):

    grid_search = GridSearchCV(model, 
                               param_grid=param_grid, cv=5, 
                               n_jobs=-1, scoring=make_scorer(f1_score))
    fit = grid_search.fit(X_train, y_train)
    predicted = fit.predict(X_test)
    return grid_search.best_params_


In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import pickle

## Predict 2017

In [128]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    sm = SMOTE()
    
    X_train, y_train = sm.fit_sample(X_train, y_train)

    return X_train, y_train, X_test, y_test
    

In [163]:
def transform_dummify(X, columns_to_keep):
    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

#     print(list(df_temp.columns.values))
    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)
    return df_temp

## Label encoder is not sufficient. Need OneHotEncoder. 

In [33]:
# def transform_data(X):
#     X['extra_people'].replace('[\=$,]', '', regex=True, inplace=True)
# #     X['price'].replace('[\=$,]', '', regex=True, inplace=True)
#     X['property_type_new'] = le.fit_transform(X['property_type_new'])
#     X['bed_type'] = le.fit_transform(X['bed_type'])
#     X['cancellation_policy'] = le.fit_transform(X['cancellation_policy'])
#     X['room_type'] = le.fit_transform(X['room_type'])
 
#     return X

In [129]:
def predict_data(X_train, y_train, X_test, y_test):
    
    param_grid = {'n_estimators': [500, 1000, 1500], 
                  'max_features': ['auto'], 
                  'max_depth': [None, 12, 6],
                  'class_weight': [None, 'balanced']}

    model = RandomForestClassifier()
    
    best_parameters = grid_searching(param_grid, model)
    
    rf = RandomForestClassifier(n_estimators = best_parameters['n_estimators'], n_jobs=-1, random_state=0, max_depth = best_parameters['max_depth'], class_weight = best_parameters['class_weight'])
    rf.fit(X_train, y_train)
    
    predicted = rf.predict(X_test)
    scores[0][model_num] = accuracy_score(y_test, predicted)
    scores[1][model_num] = recall_score(y_test, predicted)
    scores[2][model_num] = precision_score(y_test, predicted)
    scores[3][model_num] = f1_score(y_test, predicted)
    return scores, rf

In [165]:
scores = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']


##USED THIS ONE EARLIER
# columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']

columns_to_keep = ['neighbourhood_cleansed', 'review_scores_rating', 'host_response_rate', 'access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']


#THIS IS A TEST ONE
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price_per_guest','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']

In [ ]:
##DATA BELOW IS WITH TEST COLUMN TO KEEP

## SCORE WITH LABEL ENCODER

In [10]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_dummify(X_train)
    X_test = transform_dummify(X_test)
    scores, rf = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)

1 4 0
[[0.91218782 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.6209068  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.85963383 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.72102377 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.91218782 0.91204128 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.6209068  0.60176434 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.85963383 0.87614679 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.72102377 0.71348524 0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.91218782 0.91204128 0.91266226 0.         0.         0.
  0.         0.         0.        ]
 [0.6209068  0.60176434 0.62394705 0.         0.         0.
  0.         0.         0.        ]
 [0.85963383 0.87614

In [133]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_dummify(X_train, columns_to_keep)
    X_test = transform_dummify(X_test, columns_to_keep)
    scores, rf = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)

KeyError: "['neighborhood_cleansedreview_scores_rating'] not in index"

In [138]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_dummify(X_train, columns_to_keep)
    X_test = transform_dummify(X_test, columns_to_keep)
    scores, rf = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)
    break

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/m

1 4 0
[[0.94947635 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.79974811 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.91301222 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.85263511 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
CPU times: user 56 s, sys: 792 ms, total: 56.8 s
Wall time: 4min 22s


In [268]:
np.array(X_train.columns)

array(['access_filled', 'house_rules_filled', 'space_filled',
       'accommodates', 'bed_type', 'extra_people', 'price_per_guest',
       'price_per_bedroom', 'guests_included', 'host_about_filled',
       'host_picture_url_filled', 'cancellation_policy', 'room_type',
       'property_type_new'], dtype=object)

In [266]:
rf.feature_importances_

array([0.01208263, 0.01212339, 0.0138335 , 0.07966735, 0.01365546,
       0.12504533, 0.29528097, 0.2933239 , 0.01923429, 0.02446119,
       0.00006632, 0.04731615, 0.02139722, 0.04251229])

In [51]:
from tempfile import TemporaryFile
scores_rf_baseline = TemporaryFile()
np.save(scores_rf_baseline,scores)x

In [53]:
scores_rf_baseline.seek(0) # Only needed here to simulate closing & reopening file
np.load(scores_rf_baseline)

array([[0.81988506, 0.89212093, 0.88964655, 0.88999322, 0.88094975,
        0.88061452, 0.88234636, 0.88732079, 0.85575529],
       [0.2336272 , 0.47006931, 0.48648649, 0.4973822 , 0.47533875,
        0.49067661, 0.49973698, 0.53308824, 0.5445224 ],
       [0.51456311, 0.88075561, 0.87473002, 0.88601036, 0.88855117,
        0.93692777, 0.9047619 , 0.89664311, 0.86982759],
       [0.32135123, 0.61298274, 0.62524122, 0.63710879, 0.61935028,
        0.64405594, 0.64384954, 0.66864295, 0.66976435]])

In [63]:
accuracy_2017 = np.mean(scores[0])
recall_2017 = np.mean(scores[1])
precision_2017 = np.mean(scores[2])
f1_score_2017 = np.mean(scores[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)

0.9226396976584265
0.8155060019054567
0.8160847162531671
0.8152347872055196


# Logistic Regression (RF was much better)

In [49]:
from sklearn.linear_model import LogisticRegression

In [13]:
columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']

In [44]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [45]:
def transform_logistic(X, columns_to_keep):

    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column, drop_first=True)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)

    return df_temp

In [46]:
def predict_logistic(X_train, y_train, X_test, y_test):
    model = LogisticRegression(n_jobs=-1)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    beta_coef.append(model.coef_)
    scores_logit[0][model_num] = accuracy_score(y_test, prediction)
    scores_logit[1][model_num] = recall_score(y_test, prediction)
    scores_logit[2][model_num] = precision_score(y_test, prediction)
    scores_logit[3][model_num] = f1_score(y_test, prediction)
    return scores_logit, beta_coef

In [47]:
beta_coef = []
scores_logit = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4

In [22]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_logistic(X_train, columns_to_keep)
    X_test = transform_logistic(X_test, columns_to_keep)
    scores, beta_coef = predict_logistic(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores_logit)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


1 4 0
[[0.81779515 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.0779523  0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.81779515 0.81768209 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.05229994 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.48538012 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.0779523  0.09442548 0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.81779515 0.81768209 0.81311811 0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.05229994 0.08533654 0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.48538

In [50]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_logistic(X_train, columns_to_keep)
    X_test = transform_logistic(X_test, columns_to_keep)
    scores, beta_coef = predict_logistic(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores_logit)
    break

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


1 4 0
[[0.81747036 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.50757576 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.07790698 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
CPU times: user 628 ms, sys: 100 ms, total: 728 ms
Wall time: 753 ms


In [23]:
accuracy_2017 = np.mean(scores_logit[0])
recall_2017 = np.mean(scores_logit[1])
precision_2017 = np.mean(scores_logit[2])
f1_score_2017 = np.mean(scores_logit[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)

0.7946514661754123
0.09126554689101034
0.5577129954488265
0.15490514368378314


In [24]:
beta_coef

[array([[ 0.53007962,  0.67561263,  0.15006667,  0.10604996, -0.00633706,
         -1.19853999,  0.18120679, -0.40059964,  0.20662239,  0.55194041,
         -0.62838955, -0.38660583, -0.17662322,  0.09120517,  0.24184857,
         -0.08783601,  0.81886815,  0.49857129, -1.57654837, -0.51161991,
         -0.57295504, -0.21805468, -0.06755512, -0.38932237, -0.08382664,
         -0.38541583]]),
 array([[ 0.4976066 ,  0.61940049,  0.23575569,  0.13464937, -0.00675129,
         -1.5968739 ,  0.19256912, -0.56326032,  0.21077625,  0.62926188,
         -0.63483379, -0.26512533, -0.10449351,  0.24839223,  0.33377135,
         -0.03210413,  0.84458651,  0.566181  , -1.45039404, -0.35055838,
         -0.3713931 , -0.0344765 , -0.01641945, -0.24013341,  0.05059158,
         -0.23274721]]),
 array([[ 0.46440042,  0.62811662,  0.30463579,  0.16218218, -0.00760389,
         -2.00973591,  0.22392864, -0.73386785,  0.24822121,  0.33242678,
         -0.6659505 , -0.10145976, -0.04422087,  0.4338944 ,  

In [14]:
# Compute the change in odds ratio from a one unit change in each feature.

In [143]:
from math import exp
for i, coef in enumerate(model.coef_[0]):
    print("beta%d: %.5f" % (i + 1, exp(coef)))

beta1: 1.69907
beta2: 1.96524
beta3: 1.16191
beta4: 1.11188
beta5: 0.99368
beta6: 0.30163
beta7: 1.19866
beta8: 0.66992
beta9: 1.22952
beta10: 1.73662
beta11: 0.53345
beta12: 0.67936
beta13: 0.83810
beta14: 1.09549
beta15: 1.27360
beta16: 0.91591
beta17: 2.26793
beta18: 1.64637
beta19: 0.20669
beta20: 0.59952
beta21: 0.56386
beta22: 0.80408
beta23: 0.93468
beta24: 0.67752
beta25: 0.91959
beta26: 0.68017


In [ ]:
# Increasing the ____ feature by 1 point increases the chance of being popular by a factor of __ (1.00189).

In [144]:
for i, col in enumerate(list(X_train.columns.values)):
    print(i, col)

0 access_filled
1 house_rules_filled
2 space_filled
3 accommodates
4 extra_people
5 price_per_guest
6 price_per_bedroom
7 guests_included
8 host_about_filled
9 host_picture_url_filled
10 bed_type_Airbed
11 bed_type_Couch
12 bed_type_Futon
13 bed_type_Pull-out Sofa
14 bed_type_Real Bed
15 cancellation_policy_flexible
16 cancellation_policy_moderate
17 cancellation_policy_strict
18 cancellation_policy_super_strict_30
19 cancellation_policy_super_strict_60
20 room_type_Entire home/apt
21 room_type_Private room
22 room_type_Shared room
23 property_type_new_Apartment
24 property_type_new_House
25 property_type_new_Other


## XGBoost with one quarter

In [98]:
from xgboost import XGBClassifier

In [99]:
#test with first quarter
df_sf_temp = df_sf_2017.copy()
start_month = 1
end_month = 4
X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']


In [100]:
def transform_dummify(X, columns_to_keep):
    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

#     print(list(df_temp.columns.values))
    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)
    return df_temp

In [101]:
X_train = transform_dummify(X_train, columns_to_keep)

In [102]:
X_test = transform_dummify(X_test, columns_to_keep)

In [139]:
%%time
param_grid = {'learning_rate': [0.1, 0.5, 1, 1.5], 
             'max_depth': [12, 6]}

model = XGBClassifier()

best_params = grid_searching(param_grid, model)

my_model = XGBClassifier(n_jobs=-1, booster='gbtree', n_estimators=1000, learning_rate=1, max_depth = best_params['max_depth'], objective='binary:logistic')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

CPU times: user 14.2 s, sys: 536 ms, total: 14.7 s
Wall time: 1min 2s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [140]:
%%time
my_model.fit(X_train, y_train)

CPU times: user 2min 21s, sys: 120 ms, total: 2min 21s
Wall time: 47.3 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=12, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [141]:
# Add silent=True to avoid printing out updates with each cycle
predictions = my_model.predict(X_test)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [279]:
np.array(X_train.columns)

array(['access_filled', 'house_rules_filled', 'space_filled',
       'accommodates', 'extra_people', 'price_per_guest',
       'price_per_bedroom', 'guests_included', 'host_about_filled',
       'host_picture_url_filled', 'bed_type_Airbed', 'bed_type_Couch',
       'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict',
       'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60', 'room_type_Entire home/apt',
       'room_type_Private room', 'room_type_Shared room',
       'property_type_new_Apartment', 'property_type_new_House',
       'property_type_new_Other'], dtype=object)

In [157]:
my_model.feature_importances_

array([0.16658643, 0.03381867, 0.01784736, 0.01123969, 0.01231883,
       0.04210317, 0.07045972, 0.2095363 , 0.20181628, 0.00164362,
       0.02073614, 0.        , 0.00063088, 0.01240184, 0.00567795,
       0.00059768, 0.00011622, 0.00006641, 0.00634204, 0.00197566,
       0.00240732, 0.00026564, 0.00006641, 0.00174323, 0.0014776 ,
       0.00230771, 0.00039845, 0.00199226, 0.01298292, 0.00185945,
       0.00242392, 0.00263975, 0.00122856, 0.00175983, 0.0022579 ,
       0.0022745 , 0.00129497, 0.00230771, 0.00219149, 0.        ,
       0.00008301, 0.00308801, 0.00004981, 0.00431657, 0.0000166 ,
       0.00084671, 0.00019923, 0.00194246, 0.00630883, 0.0000332 ,
       0.0000332 , 0.00018262, 0.00054787, 0.00195906, 0.01490877,
       0.01929175, 0.01939136, 0.0000166 , 0.        , 0.00896518,
       0.00765361, 0.00122856, 0.02296084, 0.01680142, 0.00938024],
      dtype=float32)

In [142]:
accuracy_score(y_test, predictions)

0.9486707331108298

In [143]:
recall_score(y_test, predictions)

0.836272040302267

In [144]:
precision_score(y_test, predictions)

0.8771466314398944

In [145]:
f1_score(y_test, predictions)

0.8562217923920051

## ADA Boosting

In [80]:
from sklearn.ensemble import AdaBoostClassifier #For Classification

In [152]:
param_grid = {'learning_rate': [0.1, 0.5, 1, 1.5, 2]}
model = AdaBoostClassifier()

best_params = grid_searching(param_grid, model)

clf = AdaBoostClassifier(n_estimators = 1000, 
                         learning_rate = best_params['learning_rate']) 
clf.fit(X_train,y_train)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/m

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.5, n_estimators=1000, random_state=None)

In [153]:
prediction = clf.predict(X_test)

In [154]:
accuracy_score(y_test, prediction)

0.8567153872712625

In [155]:
recall_score(y_test, prediction)

0.4924433249370277

In [156]:
precision_score(y_test, prediction)

0.6404586404586404

In [119]:
f1_score(y_test, prediction)

0.5489346334416757

In [240]:
clf.feature_importances_ 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/tree/tree.py:508: RuntimeWarning: invalid value encountered in true_divide
  return self.tree_.compute_feature_importances()


array([           nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan, 2.95272271e-05,
                  nan,            nan,            nan,            nan,
                  nan,            nan, 3.21736089e-05, 9.45490779e-06,
                  nan,            nan,            nan,            nan,
                  nan,            nan])

## Gradient Boosting with one quarter

In [121]:
from sklearn.ensemble import GradientBoostingClassifier #For Classification

In [146]:
clf_gradient = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0)
clf_gradient.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [147]:
pred = clf_gradient.predict(X_test)

In [148]:
accuracy_score(y_test, pred)

0.924272068132121

In [149]:
recall_score(y_test, pred)

0.7657430730478589

In [150]:
precision_score(y_test, pred)

0.8095872170439414

In [151]:
f1_score(y_test, pred)

0.7870550161812296